# setup

In [1]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [2]:
import torch

In [20]:
from dataloaders.code15text import CODE as DS
from dataloaders.code15text import CODEsplit as DSsplit
from dataloaders.codetest import CODEtest
from models.join import JoinText
from models.baseline import ResnetBaseline
from runners.l1 import Runner
from hparams import BATCH_SIZE, NUM_WORKERS
from utils import json_dump

In [4]:
model_label = 'code15l1'

# init

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

database = DS()
tst_ds = CODEtest()

model = JoinText(ResnetBaseline(n_classes = 6))
model = torch.load('output/{}/{}.pt'.format(model_label, model_label))

In [6]:
runner = Runner(device = device, model = model, database = database, Split = DSsplit, model_label = model_label)

# run

In [10]:
val_dl = torch.utils.data.DataLoader(runner.val_ds, batch_size = BATCH_SIZE, 
                                     shuffle = False, num_workers = NUM_WORKERS)
tst_dl = torch.utils.data.DataLoader(tst_ds, batch_size = BATCH_SIZE, 
                                     shuffle = False, num_workers = NUM_WORKERS)

In [25]:
best_f1s, best_thresholds = runner._synthesis(val_dl, best_thresholds = None)
all_binary_results, all_true_labels, metrics_dict = runner._synthesis(tst_dl, best_thresholds)
json_dump(metrics_dict, model_label, test = True)

In [12]:
metrics_dict

{'Accuracy': [0.9794437726723095,
  0.9854897218863361,
  0.992744860943168,
  0.9915356711003628,
  0.9891172914147521,
  0.9842805320435308,
  0.9407496977025392],
 'F1 Score': [0.5641025641025641,
  0.8000000000000002,
  0.888888888888889,
  0.7999999999999999,
  0.6666666666666666,
  0.7936507936507937,
  0.964924838940587],
 'AUC ROC': [0.6964285714285714,
  0.851680142422669,
  0.9,
  0.9344173859432798,
  0.8430825930825931,
  0.8372049264454328,
  0.8510248828274294]}